In [7]:
import pandas as pd
import numpy as np
from Code.UtilityFunctions.wikidata_functions import wikidata_query, category_query, min_qid, get_all_wikidata_claims, compare_qids, categories_dict_singular, get_qid_label
from Code.UtilityFunctions.get_data_path import get_path
from rdflib import Namespace, Graph, URIRef, Literal, XSD
from rdflib.namespace import RDFS
import os
import gzip

In [8]:
location_mappings = pd.read_csv(get_path('location_mappings_expanded.csv'))

In [9]:
location_mappings

,coordinates,city_qid,cityLabel,population,county_qid,countyLabel,state_qid,stateLabel,country_qid,countryLabel
0,"-119.71,34.43",Q159288,Santa Barbara,88665.0,Q108106,Santa Barbara County,Q99,California,Q30,United States of America
1,"-90.34,38.55",Q966126,Grantwood Village,941.0,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
2,"-110.88,32.22",Q18575,Tucson,542629.0,Q58688,Pima County,Q816,Arizona,Q30,United States of America
3,"-75.16,39.96",Q1345,Philadelphia,1603797.0,Q496900,Philadelphia County,Q1400,Pennsylvania,Q30,United States of America
4,"-75.47,40.34",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11410,"-82.68,27.87",Q2151669,Pinellas Park,53093.0,Q494556,Pinellas County,Q812,Florida,Q30,United States of America
11411,"-119.8,39.32",Q40881,Carson City,58639.0,NaN,NaN,Q1227,Nevada,Q30,United States of America
11412,"-90.05,30.02",Q34404,New Orleans,383997.0,Q486231,Orleans Parish,Q1588,Louisiana,Q30,United States of America
11413,"-90.23,38.66",Q38022,St. Louis,301578.0,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America


In [10]:
location_mappings = location_mappings.dropna(subset=['city_qid']).sort_values(by=['cityLabel'])
location_mappings

,coordinates,city_qid,cityLabel,population,county_qid,countyLabel,state_qid,stateLabel,country_qid,countryLabel
4795,"-90.16,38.92",Q443852,Alton,25676.0,Q486448,Madison County,Q1204,Illinois,Q30,United States of America
6610,"-90.15,38.9",Q443852,Alton,25676.0,Q486448,Madison County,Q1204,Illinois,Q30,United States of America
2827,"-90.18,38.92",Q443852,Alton,25676.0,Q486448,Madison County,Q1204,Illinois,Q30,United States of America
5415,"-90.19,38.89",Q443852,Alton,25676.0,Q486448,Madison County,Q1204,Illinois,Q30,United States of America
337,"-90.15,38.92",Q443852,Alton,25676.0,Q486448,Madison County,Q1204,Illinois,Q30,United States of America
...,...,...,...,...,...,...,...,...,...,...
3896,"-86.23,39.92",Q971153,Zionsville,30603.0,Q499547,Boone County,Q1415,Indiana,Q30,United States of America
10451,"-86.21,39.98",Q971153,Zionsville,30603.0,Q499547,Boone County,Q1415,Indiana,Q30,United States of America
9154,"-86.29,39.92",Q971153,Zionsville,30603.0,Q499547,Boone County,Q1415,Indiana,Q30,United States of America
3256,"-86.24,39.95",Q971153,Zionsville,30603.0,Q499547,Boone County,Q1415,Indiana,Q30,United States of America


In [11]:
location_mappings.loc[location_mappings['cityLabel'].str.startswith('Q'), 'cityLabel'].unique()

array(['Q12530186', 'Q12536050', 'Q12551744', 'Q31455252', 'Q31455253',
       'Q31455254'], dtype=object)

# Create triples for locations on business ids

### Issue 1: some entities not being found, or not being mapped correctly
This is mostly because it is not being mapped to the borough or "census designated place" in wikidata, because we only look for the cities in radius of 20 km. This could possibly be handled by creating a new column searching for the borough or "census designated place" in wikidata, and then using that to create the triples.

### Issue 2: Canada entities are not correctly placed in the mapped dataframe
This is because the hierarchy is different from the US.

### Issue 3: Location is being mapped to Google API data in [create_nt_files](https://github.com/christiannielsen98/DVML-P7/blob/main/Code/create_nt_files.py)


In [12]:
# load the wikidata location mappings
location_mappings = pd.read_csv(get_path('location_mappings_expanded.csv'))
# Load the business data from yelp
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)


In [13]:
# Add "long_lat_round" column to the dataframe for mapping to wikidata
biz['long_lat_round'] = (biz["longitude"].apply(round, args=(2,)).astype(str) + "," + biz["latitude"].apply(round, args=(2,)).astype(str))

In [14]:
# Select only the columns we need
biz2 = biz[['business_id','long_lat_round', 'address', 'city', 'state']]

In [16]:
# Merge the business data with the location data on the "long_lat_round" column
biz_location_mapping_merge = biz2.merge(location_mappings, left_on='long_lat_round', right_on='coordinates', how='left')

In [42]:
# Create the triples from the merged dataframe
#TODO: replace the example namespace with the PURL namespace
schema = Namespace("https://schema.org/")
example = Namespace("https://example.org/")
wiki = Namespace("https://www.wikidata.org/entity/")

## If file exists, delete it ##
remove_files="/home/ubuntu/vol1/virtuoso/import/wikidata_location_mappings.nt.gz"
if os.path.isfile(remove_files):
    os.remove(remove_files)
else:    ## Show an error ##
    print("Error: %s file not found" % remove_files)

location_predicate = wiki + "P131" # P131 = located in the administrative territorial entity
population_predicate = wiki + "P1082" # P1082 = population
instance_of_predicate = wiki + "P31" # P31 = instance of
city_object = wiki + "Q515" # Q515 = city
county_object = wiki + "Q28575" # Q28575 = county
state_object = wiki + "Q35657" # Q35657 = U.S. state
province_object = wiki + "Q11828004" # Q11828004 = province of Canada
country_object = wiki + "Q6256" # Q6256 = country

list_of_us_states = list(wikidata_query(sparql_query="SELECT ?state WHERE{?state wdt:P31 wd:Q35657.}")['state.value'].apply(lambda x: x[31:]))
list_of_canada_provinces = list(wikidata_query(sparql_query="SELECT ?province WHERE{?province wdt:P31 wd:Q11828004.}")['province.value'].apply(lambda x: x[31:]))

triple_file = gzip.open(filename="/home/ubuntu/vol1/virtuoso/import/wikidata_location_mappings.nt.gz", mode="at", encoding="utf-8")

G = Graph()
for i in biz_location_mapping_merge.itertuples():
    if not pd.isna(i.city_qid):
        G.add((URIRef(example[i.business_id]), URIRef(location_predicate), URIRef(wiki[i.city_qid])))
        G.add((URIRef(wiki[i.city_qid]), URIRef(RDFS.label), Literal(i.cityLabel, datatype=XSD.string)))
        G.add((URIRef(wiki[i.city_qid]), URIRef(instance_of_predicate), URIRef(city_object)))
        if not pd.isna(i.population):
            G.add((URIRef(wiki[i.city_qid]), URIRef(population_predicate), Literal(i.population, datatype=XSD.integer)))
    if not pd.isna(i.county_qid):
        G.add((URIRef(example[i.business_id]), URIRef(location_predicate), URIRef(wiki[i.county_qid])))
        G.add((URIRef(wiki[i.county_qid]), URIRef(RDFS.label), Literal(i.countyLabel, datatype=XSD.string)))
        G.add((URIRef(wiki[i.county_qid]), URIRef(instance_of_predicate), URIRef(county_object)))
    if not pd.isna(i.state_qid):
        G.add((URIRef(example[i.business_id]), URIRef(location_predicate), URIRef(wiki[i.state_qid])))
        G.add((URIRef(wiki[i.state_qid]), URIRef(RDFS.label), Literal(i.stateLabel, datatype=XSD.string)))
        if i.state_qid in list_of_us_states:
            G.add((URIRef(wiki[i.state_qid]), URIRef(instance_of_predicate), URIRef(state_object)))
        elif i.state_qid in list_of_canada_provinces:
            G.add((URIRef(wiki[i.state_qid]), URIRef(instance_of_predicate), URIRef(province_object)))
    if not pd.isna(i.country_qid):
        G.add((URIRef(example[i.business_id]), URIRef(location_predicate), URIRef(wiki[i.country_qid])))
        G.add((URIRef(wiki[i.country_qid]), URIRef(RDFS.label), Literal(i.countryLabel, datatype=XSD.string)))
        G.add((URIRef(wiki[i.country_qid]), URIRef(instance_of_predicate), URIRef(country_object)))
        
triple_file.write(G.serialize(format="nt"))
triple_file.close()

Error: /home/ubuntu/vol1/virtuoso/import/wikidata_location_mappings.nt.gz file not found


# Evaluate the results

In [25]:
len(biz_location_mapping_merge['city'].drop_duplicates()), len(biz_location_mapping_merge['cityLabel'].drop_duplicates())

(1416, 245)

In [26]:
biz_location_mapping_merge[['city', 'cityLabel']].drop_duplicates()

,city,cityLabel
0,Santa Barbara,Santa Barbara
1,Affton,Grantwood Village
2,Tucson,Tucson
3,Philadelphia,Philadelphia
4,Green Lane,NaN
...,...,...
149841,Wales,NaN
150054,UPPER MORELAND,Beverly
150094,Glenside,Beverly
150183,St.Louis,Country Club Hills


In [34]:
biz_location_mapping_merge[biz_location_mapping_merge['city'] != biz_location_mapping_merge['cityLabel']][['city','cityLabel']]

,city,cityLabel
1,Affton,Grantwood Village
4,Green Lane,NaN
6,Brentwood,Richmond Heights
7,St. Petersburg,South Pasadena
8,Affton,Shrewsbury
...,...,...
150340,Edmonton,"Strathcona,"
150341,Edmonton,"Strathcona,"
150342,Nashville,Berry Hill
150343,Indianapolis,Carmel


In [40]:
pd.DataFrame([biz_location_mapping_merge['city'].value_counts(), biz_location_mapping_merge['cityLabel'].value_counts()]).T.head(50)

,city,cityLabel
Philadelphia,14569.0,15291.0
Tucson,9250.0,5648.0
Tampa,9050.0,6110.0
Indianapolis,7540.0,1743.0
Nashville,6971.0,3623.0
New Orleans,6209.0,5277.0
Reno,5935.0,5311.0
Edmonton,5054.0,2834.0
Saint Louis,4827.0,NaN
Santa Barbara,3829.0,3608.0


In [41]:
biz

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,long_lat_round
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,"-119.71,34.43"
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...","-90.34,38.55"
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...","-110.88,32.22"
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","-75.16,39.96"
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...","-75.47,40.34"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3...","-113.49,53.47"
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3...","-86.77,36.12"
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None,"-86.07,39.91"
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...","-89.95,38.78"
